In [ ]:
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
df=pd.read_csv("Real_Combine_data.csv")
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
X=df.iloc[:,:-1]#independent features
Y=df.iloc[:,-1]#dependent features

In [ ]:
X

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2
...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5


In [ ]:
Y

0       219.720833
1       182.187500
2       154.037500
3       223.208333
4       200.645833
           ...    
1088    288.416667
1089    256.833333
1090    169.000000
1091    186.041667
1092    185.583333
Name: PM 2.5, Length: 1093, dtype: float64

In [ ]:
def build_model(hp):
  model=keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):
    model.add(Dense(units=hp.Int('units_'+ str(i),
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
  model.add(Dense(1,activation='linear'))
  model.compile(optimizer=keras.optimizers.Adam(
      hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
      loss='mean_absolute_error',
      metrics=['mean_absolute_error'])
  return model

In [ ]:
tuner=RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=10)

In [ ]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 9 Complete [00h 00m 09s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 12m 19s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 9
units_0: 512
units_1: 256
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 13
units_0: 320
units_1: 416
learning_rate: 0.001
units_2: 384
units_3: 64
units_4: 288
units_5: 320
units_6: 224
units_7: 64
units_8: 192
units_9: 32
units_10: 32
units_11: 32
units_12: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 5
units_0: 32
units_1: 64
learning_rate: 0.0001
units_2: 480
units_3: 448
units_4: 288
units_5: 448
units_6: 320
units_7: 352
units_8: 352
units_9: 128
units_10: 480
units_11: 384
units_12: 96
Score: nan
Trial summary
Hyperparameters:
num_layers: 15
units_0: 352
units_1: 96
learning_rate: 0.01
units_2: 480
units_3: 256
units_4: 256
units_5: 480
units_6: 64
units_7: 352
units_8: 320
units_9